In [1]:
# Imports
import pandas as pd
import numpy as np
import tifffile as tif
import matplotlib.pyplot as plt
import os, gc, glob
import json 
from shapely import wkt
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras import Sequential
import geopandas as gdp


In [2]:
# Load image from directory
dirname = '/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/temp2/'  # absolute path
tif_files = []   # list which will contain array of individual tif files
# iterate through the file directory
for fname in os.listdir(dirname):
    print(fname)
    im = Image.open(os.path.join(dirname, fname))
    imarray = np.array(im)
    tif_files.append(imarray)

6010_1_2.tif


In [3]:
len(tif_files)

1

In [4]:
# explore the shape of the files
array_shape = []
for i in tif_files:
    print(i.shape)
    array_shape.append(i.shape)

(3349, 3396, 3)


In [5]:
# get the min and max values of the shape of the images 
df_shape = pd.DataFrame(array_shape)
df_shape.head(3)

,0,1,2
0,3349,3396,3


In [6]:
# X shape min
x_min = df_shape[0].min()
x_min

3349

In [7]:
# y shape min
y_min = df_shape[1].min()
y_min

3396

In [8]:
# reshape the individual images using a using numpy slicing

reshape_image = []
for image in tif_files:
    #print(image)
    reshape_image.append(image[:x_min,:y_min])

In [9]:
# verify the image shapes are equal 
for i in reshape_image:
    print(i.shape)

(3349, 3396, 3)


In [10]:
X_train = np.asarray(reshape_image)
X_train.shape


(1, 3349, 3396, 3)

## Merging Weiyi's Shit

In [11]:
#open geojson training folder
file = '/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/train_geojson_v3/'

for name in os.listdir(file):
    files = os.walk(file)
    geojson_files = [f[0] for f in os.walk(file)]

In [12]:
geojson_folders = os.listdir(geojson_files[0]) #list of all the folders inside the training folder
geojson_folders

['6110_4_0',
 '6070_2_3',
 '6010_1_2',
 '6040_1_0',
 '6060_2_3',
 '6100_2_3',
 '6100_2_2',
 '6170_2_4',
 '6090_2_0',
 '6110_3_1',
 '6170_4_1',
 '6110_1_2',
 '6100_1_3',
 '6120_2_0',
 '6040_4_4',
 '6150_2_3',
 '6040_1_3',
 '6010_4_2',
 '6170_0_4',
 '6010_4_4',
 '6160_2_1',
 '6140_3_1',
 '6140_1_2',
 '6040_2_2',
 '6120_2_2']

In [13]:
geojson_files[0]

'/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/train_geojson_v3/'

In [14]:
geojson_per_image = {}

for i in geojson_folders:
    fichiers = os.listdir(geojson_files[0]+ f"/{i}")
    geojson_per_image[i] = int(any('008' in s for s in fichiers)) #target water

In [15]:
geojson_per_image

{'6110_4_0': 1,
 '6070_2_3': 1,
 '6010_1_2': 0,
 '6040_1_0': 0,
 '6060_2_3': 1,
 '6100_2_3': 1,
 '6100_2_2': 1,
 '6170_2_4': 1,
 '6090_2_0': 1,
 '6110_3_1': 1,
 '6170_4_1': 0,
 '6110_1_2': 1,
 '6100_1_3': 1,
 '6120_2_0': 0,
 '6040_4_4': 0,
 '6150_2_3': 1,
 '6040_1_3': 0,
 '6010_4_2': 0,
 '6170_0_4': 1,
 '6010_4_4': 0,
 '6160_2_1': 0,
 '6140_3_1': 1,
 '6140_1_2': 1,
 '6040_2_2': 0,
 '6120_2_2': 1}

In [16]:
df_target = pd.DataFrame.from_dict(geojson_per_image, orient='index')
df_target


,0
6110_4_0,1
6070_2_3,1
6010_1_2,0
6040_1_0,0
6060_2_3,1
6100_2_3,1
6100_2_2,1
6170_2_4,1
6090_2_0,1
6110_3_1,1


In [17]:
len(df_target)

25

In [18]:
X_train.shape

(1, 3349, 3396, 3)

In [19]:
y_train = np.asanyarray([0])
y_train

array([0])

In [20]:
#tf.image.resize(
 #   images, size, method=ResizeMethod.BILINEAR, preserve_aspect_ratio=False,
  #  antialias=False, name=None
#)


In [21]:
def initialize_model():
    model = Sequential()
    model.add(layers.Conv2D(16, activation = 'relu', kernel_size=(300,300),input_shape=(3349, 3396, 3)))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    #model.add(layers.Reshape((3348,3338),input_shape=(3348, 3338, 3)))
    #model.add(layers.Conv2D(16, activation = 'relu', kernel_size=(150,150)))
    #model.add(layers.MaxPool2D(pool_size=(2,2)))          
    model.add(layers.Flatten())
    model.add(layers.Dense(8, activation = 'relu'))          
    model.add(layers.Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics='accuracy')
    #model.fit(X_train,y_train)
    return model

In [22]:
model=initialize_model()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=1)

model.fit(X_train,y_train, epochs=3)

Epoch 1/3


## Let's try try to predict something 

In [ ]:
#import tensorflow as tf
#tf.trainable_variables()

In [ ]:
# Prepare a predict

# Load image from directory
dirname = '/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/predict/'  # absolute path
predict = []   # list which will contain array of individual tif files
# iterate through the file directory
for fname in os.listdir(dirname):
    print(fname)
    im = Image.open(os.path.join(dirname, fname))
    imarray = np.array(im)
    predict.append(imarray)

In [ ]:
# explore the shape of the files
predict_shape = []
for i in predict:
    print(i.shape)
    array_shape.append(i.shape)

In [ ]:
# put the the data into a numpy array
array_pred = np.asanyarray(predict)
array_pred.shape

In [ ]:
y_test0 = np.array([0])
y_test0.shape

In [ ]:
y_test1 = np.array([1])
y_test1.shape

In [ ]:
#first trial predict

model.predict(array_pred)

In [ ]:
#array_pred.shape

In [ ]:
#X_train.shape

In [ ]:
#x_test = array_pred[0,:,:,:]
#x_test.shape

In [ ]:
#a = np.append(x_test,x_test[])


In [ ]:
#reshape_image = []

 #   reshape_image.append(image[:x_min,:y_min])

In [ ]:
pic = tif.imread('/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/sixteen_band/6010_0_1_P.tif') 
tif.imshow(pic,cmap='gray')